In [3]:
import pandas as pd
import geopandas as gpd
import os
from shapely.geometry import Point
import xarray as xr
from scipy.stats import zscore


### Clean up EPD Water Quality Data
#### 1 - Join all stations to one dataframe
#### 2- Create a dataframe of surface and bottom readings on a single row
#### 3 - Join with Station Metadata

In [4]:
# Setup file paths
water_quality_raw_dir="../data/epd_water_quality/raw" #directory containing all raw files from EPD water quality dataset
epd_stations_fp="../data/epd_water_quality/prc/epd_stations.csv"

# Read Water Quality Data from all stations and concat to one df
col_rn=['site', 'station', 'date', 'sample_id', 'depth','chla', 'diss_o', 'ph','salinity', 'turbidity', 'temp','suspended_solids', 'nitrates']
water_quality_df_list=[pd.read_csv(f"{water_quality_raw_dir}/{file}") for file in os.listdir(water_quality_raw_dir)]
water_quality_df=pd.concat(water_quality_df_list).reset_index(drop=True).dropna()
water_quality_df.rename(columns=dict(zip(water_quality_df.columns,col_rn)), inplace=True) # Rename Columns for simplicity
water_quality_df.date=pd.to_datetime(water_quality_df.date)
# Retrieve Surface and Bottom Data 
surface=water_quality_df[water_quality_df.depth=="Surface Water"]
bottom=water_quality_df[water_quality_df.depth=="Bottom Water"]

# Join surface and bottom datasets on site, station and date, resulting in a single dataframe including surface / bottom data for a single row (station timepoint)
join_cols=["site","station","date"]
water_quality_df_joined=surface.set_index(join_cols).join(bottom.set_index(join_cols), lsuffix="_surf", rsuffix="_bott").dropna().drop(columns=["depth_surf","depth_surf","sample_id_surf","sample_id_bott"]).reset_index()

# Read EPD Station Metadata
epd_stations=pd.read_csv(epd_stations_fp)
# Join station metadata with water quality df
water_quality_df_joined=water_quality_df_joined.set_index("station").join(epd_stations.set_index("station")).reset_index()

# # Convert string values to float, remove "<"
vars=['chla_surf', 'diss_o_surf', 'ph_surf',
       'salinity_surf', 'turbidity_surf', 'temp_surf', 'suspended_solids_surf',
       'nitrates_surf','chla_bott', 'diss_o_bott', 'ph_bott',
       'salinity_bott', 'turbidity_bott', 'temp_bott', 'suspended_solids_bott',
       'nitrates_bott']

for vr in vars:

    if water_quality_df_joined[vr].dtype == object:
        water_quality_df_joined[vr]=water_quality_df_joined[vr].str.replace("<","").astype(float)

# Get Temperature Delta
water_quality_df_joined["temp_delta"]= water_quality_df_joined.temp_bott - water_quality_df_joined.temp_surf 
vars.append('temp_delta')


In [5]:

#Normalize data with z-score
norm_water_quality_df=water_quality_df_joined.copy()
for vr in vars:
        norm_water_quality_df[vr]=zscore(norm_water_quality_df[vr])

norm_wq_vars=water_quality_df_joined[vars]
wq_vars=water_quality_df_joined[vars]

norm_summer=norm_water_quality_df[(norm_water_quality_df.date.dt.month > 5) & (norm_water_quality_df.date.dt.month < 10)]
norm_summer[vars][(norm_summer[vars] < 2) & (norm_summer[vars] > -2)].dropna()

# Filter Summer Data (June - Sept)
summer=water_quality_df_joined[(water_quality_df_joined.date.dt.month > 5) & (water_quality_df_joined.date.dt.month < 10)]
summer_mean=summer.groupby('station')[vars].mean().join(epd_stations.set_index("station"))

# Filter Winter Data (June - Sept)
winter=water_quality_df_joined[ (water_quality_df_joined.date.dt.month > 10) | (water_quality_df_joined.date.dt.month < 3)]
winter_mean=winter.groupby('station')[vars].mean().join(epd_stations.set_index("station"))


In [16]:
print(f"TOTAL READINGS: {len(water_quality_df_joined)}")
print("")
water_quality_df_joined.head()

29967


,station,site,date,chla_surf,diss_o_surf,ph_surf,salinity_surf,turbidity_surf,temp_surf,suspended_solids_surf,...,ph_bott,salinity_bott,turbidity_bott,temp_bott,suspended_solids_bott,nitrates_bott,latitude,longitude,depth_m,temp_delta
0,DM1,Deep Bay,1987-04-30,0.2,5.1,7.8,17.9,25.3,25.7,NaN,...,7.8,18.5,25.4,25.5,NaN,NaN,22.496150,114.010733,2,-0.2
1,DM3,Deep Bay,1986-08-07,2.7,5.7,8.1,15.5,10.0,28.7,NaN,...,8.1,18.6,15.0,28.4,NaN,NaN,22.476667,113.959183,3,-0.3
2,DM3,Deep Bay,1986-10-03,2.7,5.2,8.0,26.6,20.0,26.6,NaN,...,8.0,26.8,17.0,26.4,NaN,NaN,22.476667,113.959183,3,-0.2
3,DM3,Deep Bay,1987-04-30,9.2,7.1,8.1,21.0,4.2,25.7,NaN,...,8.1,21.3,5.4,25.4,NaN,NaN,22.476667,113.959183,3,-0.3
4,DM3,Deep Bay,1991-02-25,2.9,8.3,8.1,29.6,7.1,17.2,NaN,...,8.1,29.8,11.0,17.2,NaN,NaN,22.476667,113.959183,3,0.0


In [14]:
# Build geojson for epd_stations with mean values

s_coords=summer_mean[["latitude","longitude"]].apply(lambda x: Point(x.longitude,x.latitude), axis=1)
summer_mean_gdf=gpd.GeoDataFrame(summer_mean,geometry=s_coords,crs=4326)
summer_mean_gdf.to_file("../data/epd_water_quality/prc/epd_stations_summer_mean.geojson")
# winter_mean

## Export Data

In [24]:
## Expot

# # Write final dataset to CSV
water_quality_df_joined.dropna().to_csv("../data/epd_water_quality/prc/epd_water_quality_1986_2022.csv",index=False)
summer.dropna().to_csv("../data/epd_water_quality/prc/summer_1986_2022.csv")
summer_mean.dropna().to_csv("../data/epd_water_quality/prc/summer_mean_1986_2022.csv")
winter.dropna().to_csv("../data/epd_water_quality/prc/winter_1986_2022.csv")
winter_mean.dropna().to_csv("../data/epd_water_quality/prc/winter_mean_1986_2022.csv")